In [1]:
%matplotlib inline
import utils;
reload(utils)
from utils import *
from __future__ import division, print_function

Using TensorFlow backend.


In [2]:
path = "./data/fish/"
batch_size=64

In [3]:
batches = get_batches(path + 'train', batch_size = batch_size)
val_batches = get_batches(path+'valid', 
                          batch_size = batch_size*2, shuffle=False)
(val_classes, trn_classes, val_labels, trn_labels, val_filenames, 
 filenames, test_filenames) = get_classes(path)

Found 3697 images belonging to 8 classes.
Found 80 images belonging to 8 classes.
Found 3697 images belonging to 8 classes.
Found 80 images belonging to 8 classes.
Found 1000 images belonging to 1 classes.


In [4]:
raw_filename = [f.split('/')[-1] for f in filenames]
raw_test_filenames = [f.split('/')[-1] for f in test_filenames]
raw_val_filenames = [f.split('/')[-1] for f in val_filenames]

In [5]:
%cd /Users/trinakarmakar/anaconda2/

/Users/trinakarmakar/anaconda2


In [6]:
from Vgg16bn import Vgg16BN
model =  vgg_ft_bn(8)

Vgg16bn.py:58: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  model.add(Convolution2D(filters, 3, 3, activation='relu'))
Vgg16bn.py:58: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu")`
  model.add(Convolution2D(filters, 3, 3, activation='relu'))
Vgg16bn.py:58: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (3, 3), activation="relu")`
  model.add(Convolution2D(filters, 3, 3, activation='relu'))
Vgg16bn.py:58: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(512, (3, 3), activation="relu")`
  model.add(Convolution2D(filters, 3, 3, activation='relu'))


In [7]:
trn = get_data(path+'train')
val = get_data(path+'valid')

Found 3697 images belonging to 8 classes.
Found 80 images belonging to 8 classes.


In [ ]:
save_array(path+'results/trn.dat', trn)
save_array(path+'results/val.dat', val)

In [ ]:
test = load_array(path+'results/trn.dat')
val = load_array(path+'results/val.dat')

In [ ]:
save_array(path+'results/test.dat', test)

In [ ]:
test = load_array(path+'results/test.dat')

In [8]:
gen = image.ImageDataGenerator()

In [9]:
model.compile(optimizer=Adam(1e-3), loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
model.fit(trn, trn_labels, batch_size=batch_size, nb_epoch=1,
         validation_data = (val, val_labels))

/Users/trinakarmakar/anaconda2/lib/python2.7/site-packages/keras/models.py:848: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 3697 samples, validate on 80 samples
Epoch 1/1
3697/3697 [==============================] - 15589s - loss: 2.7524 - acc: 0.5061 - val_loss: 2.4964 - val_acc: 0.3875

In [ ]:
model.save_weights(path+'results/ft1.h5')

In [ ]:
model.load_weights(path+'results/ft1.h5')

In [ ]:
#splitting output layers and feature learning layers as output layers
# hardly require any fine tuning, we need to only fine tune conv layers
conv_layers, fc_layers = split_at(model, Convolution2D)

In [ ]:
conv_model = Sequential(conv_layers)

In [ ]:
conv_feat = conv_model.predict(trn)

In [ ]:
conv_val_feat = conv_model.predict(val)

In [ ]:
conv_test_feat = conv_model.predict(test)

In [ ]:
save_array(path+'results/conv_val_feat.dat', conv_val_feat)
save_array(path+'results/conv_feat.dat', conv_feat)

In [ ]:
save_array(path+'results/conv_test_feat.dat', conv_test_feat)

In [ ]:
conv_feat = load_array(path+'results/conv_feat.dat')
conv_val_feat = load_array(path+'results/conv_val_feat.dat')

In [ ]:
conv_test_feat = load_array(path+'results/conv_test_feat.dat')

In [ ]:
conv_val_feat.shape

In [ ]:
#Lets create a baseline model

def get_bn_layers(p):
    return [
        MaxPooling2D(inut_shape=conv_layers[-1].output_shape[1:]),
        BatchNormalization(axis=1),
        Dropout(p/4),
        Flatten(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(p/2),
        Dense(8, activation='softmax')
    ]

In [ ]:
p = 0.6

In [ ]:
bn_model = Sequential(get_bn_layers(p))

In [ ]:
bn_model.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
bn_model.fit(conv_feat, trn_labels, batch_size=batch_szie, nb_epoch=3,
            validation_data=(conv_val_feat, val_labels))

In [ ]:
bn_model.save_weights(path+'models/conv_512_6.h5

In [ ]:
bn_model.evaluate(conv_val_feat, val_labels)

In [ ]:
bn_model.load_weights(path+'models/conv_512_6.h5')

In [ ]:
#create list of image file sizes as there are files with different file sizes
sizes = [PIL.Image.open(path+'train/'+f).size for f in filenames]
id2size = list(set(sizes))
size2id = {o:i for i,o in enumerate(id2size)}

In [ ]:
import collections
collections.Counter(sizes)

In [ ]:
trn_sizes_orig = to_categorical([size2id[o] for o in sizes], len(id2sizes))

In [ ]:
raw_val_sizes = [PIL.Image.open(path+'valid/'+f).size for f in val_filenames]
val_sizez = to_categorical([sizeid[o] for o in raw_val_sizes], len(id2size))

In [ ]:
trn_sizes = trn_sizes_orig-trn_sizes_orig.mean(axis=0)/trn_sizes_orig.std(axis=0)
val_sizes = val_sizes-trn_sizes_orig.mean(axis=0)/trn_sizes_orig.std(axis=0)

In [ ]:
p = 0.6

In [ ]:
inp = Input(conv_layers[-1].output_shape[1:])
sz_inp = Input((len(id2size),))
bn_inp = BatchNormalization()(sz_inp)
x = MaxPooling2D()(inp)
x = BatchNormalization(axis=1)(x)
x = Dropout(p/4)(x)
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(p)(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(p/2)(x)
x = merge([x,bn_inp], 'concat')
x = Dense(8, activation='softmax')(x)

In [ ]:
model = Model([inp, sz_inp], x)
model.compile(Adam(lr=0.001), loss = 'categotical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit([conv_feat, trn_sizes], trn_labels, batch_size=batch_size, nb_epoch=3,
         validation_data=([conv_val_feat, val_sizes], val_labels))

In [ ]:
bn_model.optimizer.lr = 1e-4

In [ ]:
model.fit([conv_feat, trn_sizes], trn_labels, batch_size=batch_size, nb_epoch=3,
         validation_data=([conv_val_feat, val_sizes], val_labels))

In [ ]:
import ujson as json

In [ ]:
anno_classes = ['alb', 'bet', 'dol', 'lag', 'other', 'shark', 'yft']

In [ ]:
def get_annotations():
    annot_urls = {
        '5458/bet_labels.json': 'bd20591439b650f44b36b72a98d3ce27',
        '5459/shark_labels.json': '94b1b3110ca58ff4788fb659eda7da90',
        '5460/dol_labels.json': '91a25d29a29b7e8b8d7a8770355993de',
        '5461/yft_labels.json': '9ef63caad8f076457d48a21986d81ddc',
        '5462/alb_labels.json': '731c74d347748b5272042f0661dad37c',
        '5463/lag_labels.json': '92d75d9218c3333ac31d74125f2b380a'
    }
    cache_subdir = os.path.abspath(os.path.join(path, 'annos'))
    url_prefix = 'https://kaggle2.blob.core.windows.net/forum-message-attachments/147157/'
    
    if not os.path.exists(cache_subdir):
        os.makedirs(cache_subdir)
    
    for url_suffix, md5_hash in annot_urls.iteritems():
        fname = url_suffix.rsplit('/', 1)[-1]
        get_file(fname, url_prefix + url_suffix, cache_subdir=cache_subdir, md5_hash=md5_hash)

In [ ]:
get_annotations()

In [ ]:
bb_json = {}
for c in anno_classes:
    if c == 'other': continue # no annotation file for "other" class
    j = json.load(open('{}annos/{}_labels.json'.format(path, c), 'r'))
    for l in j:
        if 'annotations' in l.keys() and len(l['annotations'])>0:
            bb_json[l['filename'].split('/')[-1]] = sorted(
                l['annotations'], key=lambda x: x['height']*x['width'])[-1]

In [ ]:
bb_json['img_04908.jpg']

In [ ]:
file2idx = {o:i for i,o in enumerate(raw_filenames)}
val_file2idx = {o:i for i,o in enumerate(raw_val_filenames)}

In [ ]:
empty_bbox = {'height': 0., 'width': 0., 'x': 0., 'y': 0.} #for images that doesn't have any annotations

In [ ]:
for f in raw_filenames:
    if not f in bb_json.keys():
        bb_json[f] = empty_bbox
for f in raw_val_filenames:
    if not f in bb_json.keys():
        bb_json[f] = empty_bbox

In [ ]:
bb_params = ['height', 'width', 'x', 'y']
def convert_bb(bb, size):
    bb = [bb[p] for p in bb_params]
    conv_x = (224. / size[0])
    conv_y = (224. / size[1])
    bb[0] = bb[0]*conv_y
    bb[1] = bb[1]*conv_x
    bb[2] = max(bb[2]*conv_x, 0)
    bb[3] = max(bb[3]*conv_y, 0)
    return bb

In [ ]:
trn_bbox = np.stack([convert_bb(bb_json[f], s) for f,s in zip(raw_filenames, sizes)], 
                   ).astype(np.float32)
val_bbox = np.stack([convert_bb(bb_json[f], s) 
                   for f,s in zip(raw_val_filenames, raw_val_sizes)]).astype(np.float32)

In [ ]:
def create_rect(bb, color='red'):
    return plt.Rectange(bb[2], bb[3], bb[1], bb[0], color=color, fill=False, lw=3)

def show_bb(i):
    bb = val_bbox[i]
    plot(val[i])
    plt.gca().add_patch(create_rect(bb))